In [169]:
import pandas as pd
import sqlalchemy as sq
from sqlalchemy import text
import pyodbc as po
import pymysql as pm
import datetime as da
from datetime import date as dt
from datetime import timedelta as td

In [170]:
SERVER = 'SEVENTH\INSIDEINS'
DATABASE = 'Service'
DRIVER = 'SQL Server Native Client 11.0'
USERNAME = 'controladoria'
PASSWORD = 'Brb1508.'
DATABASE_CONNECTION = f'mssql://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver={DRIVER}'

engine = sq.create_engine(DATABASE_CONNECTION)
connection = engine.connect()

<>:1: SyntaxWarning: invalid escape sequence '\I'
<>:1: SyntaxWarning: invalid escape sequence '\I'
C:\Users\Controladoria\AppData\Local\Temp\ipykernel_7996\3766686350.py:1: SyntaxWarning: invalid escape sequence '\I'
  SERVER = 'SEVENTH\INSIDEINS'


In [171]:
data = (dt.today() - td(10)).isoformat()
#data = data.strftime("%Y-%m-%d")
#data = da.datetime(data.year,data.month,data.day).timestamp()
#data = da.datetime(data)
#data = da.datetime(data.year,data.month,data.day)
#data = da.datetime.fromisoformat(data.isoformat())
data

'2024-06-11'

In [172]:
ContasReceber = pd.read_sql_query(f"SELECT * FROM [Service].[dbo].[ContasReceber] where Pago = 0",connection,parse_dates=['date'])

In [173]:
intervalo = ContasReceber['Vencimento'] <= data
ContasReceber = ContasReceber.loc[intervalo]

In [174]:
ContasReceber

,Empresa,Operação,Cliente,NotaFiscal,Série,DataEmissão,FormaPagamento,Pago,Número,Letra,...,EliminadaPJ,CodFatLote,CodMarketingVei,ErroCartão,IDWakeUp,WakeEmpresaRegistro,PixUrl,PixTxId,PixEmv,CodRestriçãoInnove
0,1001,Grupo,856.0,1608,TMP,2021-01-25,31.0,False,1608,A1,...,False,NaN,None,None,None,None,None,None,None,None
1,1001,Grupo,364.0,458,REC,2019-08-28,31.0,False,458,A2,...,False,NaN,None,None,None,None,None,None,None,None
2,1001,Grupo,364.0,458,REC,2019-08-28,31.0,False,458,A3,...,False,NaN,None,None,None,None,None,None,None,None
3,1001,Grupo,244.0,544,REC,2019-11-01,31.0,False,544,A2,...,False,NaN,None,None,None,None,None,None,None,None
4,1001,Grupo,244.0,544,REC,2019-11-01,31.0,False,544,A3,...,False,NaN,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408,1001,FatLote,1279.0,4385,REC,2023-02-24,31.0,False,4385,01,...,False,946.0,None,None,None,None,None,None,None,None
1410,1001,FatLote,1566.0,6774,TMP,2023-03-21,31.0,False,6774,01,...,False,971.0,None,None,None,None,None,None,None,None
1411,1001,Grupo,1566.0,976,S,2023-03-21,31.0,False,976,A1,...,False,NaN,None,None,None,None,None,None,None,None
1412,1001,Grupo,1479.0,4509,REC,2023-03-21,31.0,False,4509,A1,...,False,NaN,None,None,None,None,None,None,None,None


In [175]:
AReceber = ContasReceber.filter(['CodInterno','Empresa','CentroResultados','Cliente','DataEmissão','Vencimento','Saldo'])

In [176]:
AReceber = AReceber.rename(columns={'CentroResultados':'CodReceita','Cliente':'CodCliente','DataEmissão':'Emissao'})

In [177]:
AReceber = AReceber.astype({'CodInterno':'int64','Empresa':'int64','CodReceita':'int64','CodCliente':'int64','Emissao':'datetime64[ns]','Vencimento':'datetime64[ns]','Saldo':'float64'})
AReceber.dtypes

CodInterno             int64
Empresa                int64
CodReceita             int64
CodCliente             int64
Emissao       datetime64[ns]
Vencimento    datetime64[ns]
Saldo                float64
dtype: object

In [178]:
for i, row in AReceber.iterrows():
    if row['Empresa'] == 1001:
        AReceber.loc[i,'Empresa'] = 1

for i, row in AReceber.iterrows():
    if row['Empresa'] == 1002:
        AReceber.loc[i,'Empresa'] = 2

for i, row in AReceber.iterrows():
    if row['Empresa'] == 1003:
        AReceber.loc[i,'Empresa'] = 3

In [179]:
emp1 = [43,47,35,7,19,93,22,17,16,8,166]

for x in emp1:
    for i, row in AReceber.iterrows():
        if row['CodReceita'] == x:
            AReceber.loc[i,'Empresa'] = 1

In [ ]:
Vencimentos = []
ven = []
Atraso = []
hoje = dt.today()
for i in AReceber['Vencimento']:
    Vencimentos.append(i)
for j in Vencimentos:
    j = str(j)
    j = da.datetime.strptime(j,'%Y-%m-%d %H:%M:%S')
    j = dt(j.year,j.month,j.day)
    ven.append(j)

for z in ven:
    Atraso.append((hoje - z).days)

In [206]:
AReceber['Atraso'] = Atraso

In [210]:
taxa = 0.1 / 100
juros = []
for a in range(len(AReceber['Saldo'])):
    k = AReceber['Saldo'][a] * AReceber['Atraso'][a] * taxa
juros

KeyError: 18

In [ ]:
AReceber

In [207]:
AReceber.to_excel("Inadimplentes.xlsx",index=False)